In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Wed Nov 30 18:45:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Benchmark 1: Lud

##CUDA

In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lud
base  common  cuda  Makefile  README  run  tools


In [4]:
!make clean
!make

cd cuda; make clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lud/cuda'
rm -f lud_cuda ../common/common.o lud.o lud_kernel.o *.linkinfo
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lud/cuda'
cd cuda; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lud/cuda'
nvcc  -I../common -O3 -use_fast_math -arch=sm_75 -lm  -DGPU_TIMER  -o ../common/common.o -c ../common/common.c
../common/common.c: In function ‘create_matrix_from_file’:
../common/common.c:51:3: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
   fscanf(fp, "%d\n", &size);
   ^~~~~~~~~~~~~~~~~~~~~~~~~
../common/common.c:61:11: warning: ignoring return value of ‘fscanf’, declared with attribute warn_unused_result [-Wunused-result]
           fscanf(fp, "%f ", m+i*size+j);
           ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [5]:
!./cuda/lud_cuda -s 2500

WG size of kernel = 16 X 16
Generate input matrix internally, size =2500
Creating matrix internally size=2500
Time consumed(ms): 94.997000


##OMP

In [6]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud
base  common  Makefile	omp  README  run  run_offload  tools


In [7]:
!make clean
!make

cd omp; make clean ; make -f Makefile.offload clean
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud/omp'
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud/omp'
rm --force lud_omp_offload lud.o lud_omp.o ../common/common.o *~
make[1]: Leaving directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud/omp'
cd omp; make
make[1]: Entering directory '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lud/omp'
gcc  -c -Wall -O3   -fopenmp -I../common lud.c -o lud.o
gcc  -c -Wall -O3   -fopenmp -I../common lud_omp.c -o lud_omp.o
gcc  -c -Wall -O3   -fopenmp -I../common ../common/common.c -o ../common/common.o
../common/common.c: In function ‘create_matrix_from_file’:

In [8]:
!./omp/lud_omp -s 2500

Generate input matrix internally, size =2500
Creating matrix internally size=2500
running OMP on host
Time consumed(ms): 1300.342000


# Benchmark 2: Myocyte

##CUDA

In [9]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/myocyte'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/myocyte
define.c		    kernel_cam.cu    main.cu	  run
embedded_fehlberg_7_8_2.cu  kernel.cu	     main.o	  solver_2.cu
embedded_fehlberg_7_8.cu    kernel_ecc_2.cu  Makefile	  solver.cu
file.c			    kernel_ecc.cu    master.cu	  timer.c
kernel_2.cu		    kernel_fin_2.cu  myocyte.out  work_2.cu
kernel_cam_2.cu		    kernel_fin.cu    README	  work.cu


In [10]:
!make clean
!make

rm *.o *.out output.txt
rm: cannot remove 'output.txt': No such file or directory
Makefile:43: recipe for target 'clean' failed
make: *** [clean] Error 1
nvcc	main.cu \
			-c -O3 -g
file.c(108): warning: result of call is not used

file.c(116): warning: result of call is not used

work.cu(126): warning: conversion from a string literal to "char *" is deprecated

work.cu(135): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(171): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(185): warning: conversion from a string literal to "char *" is deprecated

file.c(108): warning: result of call is not used

file.c(116): warning: result of call is not used

work.cu(126): warning: conversion from a string literal to "char *" is deprecated

work.cu(135): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(171): warning: conversion from a string literal to "char *" is deprecated

work_2.cu(185): warning: conver

In [11]:
!ls

define.c		    kernel_cam.cu    main.cu	  run
embedded_fehlberg_7_8_2.cu  kernel.cu	     main.o	  solver_2.cu
embedded_fehlberg_7_8.cu    kernel_ecc_2.cu  Makefile	  solver.cu
file.c			    kernel_ecc.cu    master.cu	  timer.c
kernel_2.cu		    kernel_fin_2.cu  myocyte.out  work_2.cu
kernel_cam_2.cu		    kernel_fin.cu    README	  work.cu


In [12]:
!./myocyte.out 2000 200 1

Time spent in different stages of the application:
0.000000000000 s, 0.000000000000 % : SETUP VARIABLES
0.229387000203 s, 1.395832061768 % : ALLOCATE CPU MEMORY AND GPU MEMORY
0.316181987524 s, 1.923984050751 % : READ DATA FROM FILES, COPY TO GPU MEMORY
0.000023000001 s, 0.000139956217 % : RUN GPU KERNEL
15.881480216980 s, 96.639648437500 % : COPY GPU DATA TO CPU MEMORY
0.006639000028 s, 0.040398664773 % : FREE MEMORY
Total time:
16.433710098267 s


##OMP

In [13]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/myocyte'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/myocyte
cam.c	  embedded_fehlberg_7_8.c  main.c    master.c	  run
define.c  file.c		   main.o    myocyte.out  solver.c
ecc.c	  fin.c			   Makefile  README	  timer.c


In [14]:
!make clean
!make

rm *.o myocyte.out output.txt
rm: cannot remove 'output.txt': No such file or directory
Makefile:32: recipe for target 'clean' failed
make: *** [clean] Error 1
gcc	main.c \
		-c -O3 -fopenmp
main.c: In function ‘main’:
main.c:175:20: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
    printf("ERROR: %d is the incorrect end of simulation interval, use numbers > 0\n", xmax);
                   ~^
                   %ld
main.c:185:20: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘long int’ [-Wformat=]
    printf("ERROR: %d is the incorrect number of instances of simulation, use numbers > 0\n", workload);
                   ~^
                   %ld
main.c:363:27: warning: unknown conversion type character ‘:’ in format [-Wformat=]
  printf("%.12f s, %.12f % : SETUP VARIABLES, READ COMMAND LINE ARGUMENTS\n",  (float) (time1-time0) / 1000000, (float) (time1-time0) / (float) (time5-time0) * 100);
            

In [15]:
!ls

cam.c	  embedded_fehlberg_7_8.c  main.c    master.c	  run
define.c  file.c		   main.o    myocyte.out  solver.c
ecc.c	  fin.c			   Makefile  README	  timer.c


In [16]:
!./myocyte.out 2000 200 1 0

The file was not opened for reading
Time spent in different stages of the application:
0.000007000000 s, 0.000011827727 % : SETUP VARIABLES, READ COMMAND LINE ARGUMENTS
0.074720002711 s, 0.126252546906 % : ALLOCATE MEMORY
0.260780990124 s, 0.440635204315 % : READ DATA FROM FILES
58.797946929932 s, 99.349441528320 % : RUN COMPUTATION
0.049511000514 s, 0.083657518029 % : FREE MEMORY
Total time:
59.182968139648 s


#Benchmark 3: lavaMD

## CUDA

In [17]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/cuda/lavaMD
kernel	lavaMD	main.c	main.h	main.o	makefile  README  result.txt  run  util


In [18]:
!make

gcc	 main.o \
		./kernel/kernel_gpu_cuda_wrapper.o \
		./util/num/num.o \
		./util/timer/timer.o \
		./util/device/device.o \
		-lm \
		-L/usr/local/cuda/lib64 \
		-lcuda -lcudart \
		-lgomp \
		-o lavaMD


In [19]:
!./lavaMD -boxes1d 15

thread block size of kernel = 128 
Configuration used: boxes1d = 15
Time spent in different stages of GPU_CUDA KERNEL:
 0.230206996202 s, 33.199069976807 % : GPU: SET DEVICE / DRIVER INIT
 0.000414999988 s,  0.059848807752 % : GPU MEM: ALO
 0.005950000137 s,  0.858073234558 % : GPU MEM: COPY IN
 0.453312009573 s, 65.373931884766 % : GPU: KERNEL
 0.002551000100 s,  0.367889881134 % : GPU MEM: COPY OUT
 0.000979000004 s,  0.141185492277 % : GPU MEM: FRE
Total time:
0.693413972855 s


##OMP

In [20]:
%cd '/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lavaMD'
!ls

/content/drive/MyDrive/Colab Notebooks/AppliedGPU_HW2/rodinia_3.1/openmp/lavaMD
kernel	lavaMD	main.c	main.h	main.o	makefile  README  run  util


In [21]:
!make

gcc	main.o \
		./kernel/kernel_cpu.o \
		./util/num/num.o \
		./util/timer/timer.o \
		-lm \
		-lgomp \
		-o lavaMD


In [24]:
!./lavaMD -cores 1 -boxes1d 15

Configuration used: cores = 1, boxes1d = 15
Time spent in different stages of CPU/MCPU KERNEL:
 0.000000000000 s,  0.000000000000 % : CPU/MCPU: VARIABLES
 0.000016000000 s,  0.000059282662 % : MCPU: SET DEVICE
 0.000001000000 s,  0.000003705166 % : CPU/MCPU: INPUTS
26.989324569702 s, 99.999931335449 % : CPU/MCPU: KERNEL
Total time:
26.989341735840 s
